In [ ]:
# Import libraries. You may or may not use all of these.
# !pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  # %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
# !wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
X = dataset.drop('expenses', axis=1)
y = dataset['expenses']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sex_encoder = OrdinalEncoder(categories=[['female', 'male']])
smoker_encoder = OrdinalEncoder(categories=[['no', 'yes']])
region_encoder = OrdinalEncoder(categories=[['northeast', 'northwest', 'southeast', 'southwest']])

X_train['sex'] = sex_encoder.fit_transform(X_train[['sex']])
X_train['smoker'] = smoker_encoder.fit_transform(X_train[['smoker']])
X_train['region'] = region_encoder.fit_transform(X_train[['region']])

X_test['sex'] = sex_encoder.transform(X_test[['sex']])
X_test['smoker'] = smoker_encoder.transform(X_test[['smoker']])
X_test['region'] = region_encoder.transform(X_test[['region']])

In [ ]:
import seaborn as sns
train_dataframe = pd.concat([X_train, y_train], axis=1)
test_dataframe = pd.concat([X_test, y_test], axis=1)
dataframe = pd.concat([train_dataframe, test_dataframe], axis=0)
sns.heatmap(dataframe.corr(), annot=True)

In [ ]:
train_dataset = X_train
train_labels = y_train
test_dataset = X_test
test_labels = y_test

In [ ]:
train_dataset.drop(['region', 'sex', 'children'], axis=1, inplace=True)
test_dataset.drop(['region', 'sex', 'children'], axis=1, inplace=True)

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(train_dataset.shape[1],)),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mae',
    metrics=['mae', 'mse']
)

model.summary()

In [ ]:
history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
